In [9]:
import pandas as pd 
import matplotlib.pyplot as plt
import os 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  RobustScaler
import pickle
import tensorflow as tf
import numpy as np
import gc
from gcvit import GCViTBase
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate


In [2]:
study_name = '426_convnextbase_003_998_1'

pickle_file_path = f'./data/test_{study_name}.pickle'

with open(pickle_file_path, 'rb') as f:
    test_df = pickle.load(f)

pickle_file_path = f'./data/train_{study_name}.pickle'

with open(pickle_file_path, 'rb') as f:
    train_df = pickle.load(f)

In [3]:
pd.set_option('display.max_row', None) 
pd.set_option('display.max_columns', None) 

In [ ]:
train_df.head()

In [4]:
this_study_name = '511_cgvit_3'

In [ ]:
# TODO CGViT jossa global average ja gloval maxpooling lopussa yhteen tökätty. 

In [7]:
base_model = GCViTBase(pretrain=True, input_shape=(480, 480, 3), resize_query=True)

for layer in base_model.layers:
    print(f'Layer {layer.name}')

feature_layer = base_model.get_layer('pool')

avg_pool = GlobalAveragePooling2D()(feature_layer.output)
max_pool = GlobalMaxPooling2D()(feature_layer.output)

output = Concatenate([avg_pool, max_pool])

model = tf.keras.Model(base_model.input, output)

for layer in model.layers:
    print(f'Layer {layer.name}')
    print(f'Layer {layer}')

2024-05-11 13:52:25.133250: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-11 13:52:25.194512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-11 13:52:25.194577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-11 13:52:25.198367: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-11 13:52:25.198436: I tensorflow/compile

Layer patch_embed
Layer pos_drop
Layer levels_0
Layer levels_1
Layer levels_2
Layer levels_3
Layer norm
Layer pool
Layer head


In [7]:


batch_size = 16
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Initialize the GCViT model
base_model = GCViTBase(pretrain=True, input_shape=(480, 480, 3), resize_query=True)

# Define the pooling layers
global_avg_pooling = tf.keras.layers.GlobalAveragePooling2D()
global_max_pooling = tf.keras.layers.GlobalMaxPooling2D()
concatenate_layer = tf.keras.layers.Concatenate()

def load_and_preprocess_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    return img

def extract_features_batch(image_paths):
    img_batch = np.stack([load_and_preprocess_image(img_path) for img_path in image_paths])
    features = base_model.predict(img_batch, batch_size=batch_size)
    
    # Apply pooling
    avg_pooled = global_avg_pooling(features)
    max_pooled = global_max_pooling(features)
    
    # Concatenate features
    concatenated_features = concatenate_layer([avg_pooled, max_pooled])
    return concatenated_features

# Assume 'train_df' is your DataFrame containing image paths
image_paths = train_df['image_path'].values

features_list = []
for i in range(0, len(image_paths), batch_size):
    batch_paths = image_paths[i:i+batch_size]
    batch_features = extract_features_batch(batch_paths)
    features_list.append(batch_features)
    if i // batch_size % 30 == 0:  # Clear session every 30 batches
        tf.keras.backend.clear_session()
        print(f'Clearing session')

# Stack all batch features
all_features = np.vstack(features_list)
# Assuming you want to store these features as a list in your DataFrame
train_df['features'] = list(all_features)

print("Feature extraction complete.")


TypeError: ('Keyword argument not understood:', 'include_top')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
import pickle

# Specify the file path to save the pickle file
pickle_file_path = './data/train_convnextbase_df_003_998.pickle'

# Save the train_df dataframe as a pickle file
with open(pickle_file_path, 'wb') as f:
    pickle.dump(train_df, f)


In [ ]:

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


base_model = ConvNeXtBase(weights='imagenet', include_top=False, pooling='avg')
base_model.trainable = False

def load_and_preprocess_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    
    return img


def extract_features_batch(image_paths):
    img_batch = np.stack([load_and_preprocess_image(img_path) for img_path in image_paths])
    features = base_model.predict(img_batch)        
    return features

image_paths = test_df['image_path'].values

features_list = []
j = 0
for i in range(0, len(image_paths), batch_size):
    batch_paths = image_paths[i:i+batch_size]
    batch_features = extract_features_batch(batch_paths)
    features_list.append(batch_features)
    j += 1
    if j % 30 == 0:
        tf.keras.backend.clear_session()
        gc.collect()
        print(f'Clearing session')

all_features = np.vstack(features_list)
test_df['features_avg'] = list(all_features)




In [ ]:
print(test_df.shape)
print(test_df.head())

In [ ]:
pickle_file_path = './data/test_convnextbase_df_003_998.pickle'

# # Save the train_df dataframe as a pickle file
with open(pickle_file_path, 'wb') as f:
    pickle.dump(test_df, f)

test_df.head()